In [ ]:
from session_start import get_spark

spark = get_spark("us-accidents-eda")

df = spark.read.csv(
    "s3a://us-accidents-dashboard-1445/US_Accidents_March23_sampled_500k.csv",
    header=True,
    inferSchema=True
)

df.printSchema()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/07 10:34:50 WARN Utils: Your hostname, shousenjotekimac-pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.179 instead (on interface en0)
26/01/07 10:34:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/josiejiang/anaconda3/envs/us_accidents_dashboard/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/josiejiang/.ivy2_spark/cache
The jars for the packages stored in: /Users/josiejiang/.ivy2_spark/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-47656a96-3a97-48f8-81e3-3916be86ee24;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.6 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.367 in 

500000

In [1]:
from session_start import get_spark

spark = get_spark("us-accidents-eda")

print(spark.sparkContext._jsc.sc().listJars())


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/07 10:40:18 WARN Utils: Your hostname, shousenjotekimac-pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.179 instead (on interface en0)
26/01/07 10:40:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/josiejiang/anaconda3/envs/us_accidents_dashboard/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/josiejiang/.ivy2_spark/cache
The jars for the packages stored in: /Users/josiejiang/.ivy2_spark/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-436fda6b-0291-488f-ab1b-f4d63d0c3b89;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.6 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.367 in 

List(spark://192.168.0.179:65438/jars/org.apache.hadoop_hadoop-aws-3.3.6.jar, spark://192.168.0.179:65438/jars/org.wildfly.openssl_wildfly-openssl-1.1.3.Final.jar, spark://192.168.0.179:65438/jars/com.amazonaws_aws-java-sdk-bundle-1.12.367.jar)


In [2]:
from pyspark.sql import functions as F

df_state_cnt = (
    df
    .filter(F.col("State").isNotNull())
    .groupBy("State")
    .count()
    .orderBy(F.desc("count"))
)

df_state_cnt.show()

+-----+------+
|State| count|
+-----+------+
|   CA|113274|
|   FL| 56710|
|   TX| 37355|
|   SC| 24737|
|   NY| 22594|
|   NC| 21750|
|   VA| 19515|
|   PA| 19351|
|   MN| 12333|
|   OR| 11559|
|   AZ| 11150|
|   GA| 11079|
|   IL| 10904|
|   TN| 10850|
|   MI| 10518|
|   LA|  9651|
|   MD|  9128|
|   NJ|  9020|
|   OH|  7678|
|   WA|  7004|
+-----+------+
only showing top 20 rows


In [3]:
out_state = "s3a://us-accidents-dashboard-1445/processed/state_counts"

df_state_cnt.write.mode("overwrite").parquet(out_state)


26/01/07 10:31:13 WARN Base64: JAXB is unavailable. Will fallback to SDK implementation which may be less performant.If you are using Java 9+, you will need to include javax.xml.bind:jaxb-api as a dependency.


In [3]:
print("Spark:", spark.version)
print("Hadoop:", spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion())

Spark: 4.1.0
Hadoop: 3.4.2


In [4]:
df_check = spark.read.parquet(out_state)
df_check.show()

26/01/07 10:31:24 ERROR Executor: Exception in task 0.0 in stage 17.0 (TID 39)
org.apache.spark.SparkException: [FAILED_READ_FILE.NO_HINT] Encountered error while reading file s3a://us-accidents-dashboard-1445/processed/state_counts/part-00000-0eeecc7e-92d7-4887-a310-8ce973563075-c000.snappy.parquet.  SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:911)
	at org.apache.spark.sql.execution.datasources.v2.FileDataSourceV2$.attachFilePath(FileDataSourceV2.scala:142)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:141)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:773)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at 

Py4JJavaError: An error occurred while calling o72.showString.
: org.apache.spark.SparkException: [FAILED_READ_FILE.NO_HINT] Encountered error while reading file s3a://us-accidents-dashboard-1445/processed/state_counts/part-00000-0eeecc7e-92d7-4887-a310-8ce973563075-c000.snappy.parquet.  SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:911)
	at org.apache.spark.sql.execution.datasources.v2.FileDataSourceV2$.attachFilePath(FileDataSourceV2.scala:142)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:141)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:773)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spa

In [2]:
import os
print("PYSPARK_SUBMIT_ARGS =", os.environ.get("PYSPARK_SUBMIT_ARGS"))
print("SPARK_SUBMIT_OPTS =", os.environ.get("SPARK_SUBMIT_OPTS"))


PYSPARK_SUBMIT_ARGS = --packages org.apache.hadoop:hadoop-aws:3.3.6,com.amazonaws:aws-java-sdk-bundle:1.12.367 --conf spark.jars.ivy=/Users/josiejiang/.ivy2_spark --conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem --conf spark.hadoop.fs.s3a.aws.credentials.provider=com.amazonaws.auth.DefaultAWSCredentialsProviderChain --conf spark.hadoop.fs.s3a.threads.keepalivetime=60000 --conf spark.hadoop.fs.s3a.multipart.purge.age=86400000 --conf spark.hadoop.fs.s3a.retry.interval=500 --conf spark.hadoop.fs.s3a.retry.throttle.interval=100 --conf spark.hadoop.fs.s3a.connection.ttl=300000 --conf spark.hadoop.fs.s3a.connection.establish.timeout=60000 --conf spark.hadoop.fs.s3a.connection.timeout=60000 --conf spark.hadoop.fs.s3a.socket.timeout=60000 pyspark-shell
SPARK_SUBMIT_OPTS = None


In [3]:
import os

# 把 hadoop-aws 版本从 3.3.6 替换为 3.4.2（与你的 Hadoop 3.4.2 对齐）
os.environ["PYSPARK_SUBMIT_ARGS"] = os.environ["PYSPARK_SUBMIT_ARGS"].replace(
    "org.apache.hadoop:hadoop-aws:3.3.6",
    "org.apache.hadoop:hadoop-aws:3.4.2"
)

print(os.environ["PYSPARK_SUBMIT_ARGS"])


--packages org.apache.hadoop:hadoop-aws:3.4.2,com.amazonaws:aws-java-sdk-bundle:1.12.367 --conf spark.jars.ivy=/Users/josiejiang/.ivy2_spark --conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem --conf spark.hadoop.fs.s3a.aws.credentials.provider=com.amazonaws.auth.DefaultAWSCredentialsProviderChain --conf spark.hadoop.fs.s3a.threads.keepalivetime=60000 --conf spark.hadoop.fs.s3a.multipart.purge.age=86400000 --conf spark.hadoop.fs.s3a.retry.interval=500 --conf spark.hadoop.fs.s3a.retry.throttle.interval=100 --conf spark.hadoop.fs.s3a.connection.ttl=300000 --conf spark.hadoop.fs.s3a.connection.establish.timeout=60000 --conf spark.hadoop.fs.s3a.connection.timeout=60000 --conf spark.hadoop.fs.s3a.socket.timeout=60000 pyspark-shell


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("us-accidents-s3").getOrCreate()

print("Spark:", spark.version)
print("Hadoop:", spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion())
print("Jars:", spark.sparkContext._jsc.sc().listJars())

Spark: 4.1.0
Hadoop: 3.4.2
Jars: List(spark://192.168.0.179:65438/jars/org.apache.hadoop_hadoop-aws-3.3.6.jar, spark://192.168.0.179:65438/jars/org.wildfly.openssl_wildfly-openssl-1.1.3.Final.jar, spark://192.168.0.179:65438/jars/com.amazonaws_aws-java-sdk-bundle-1.12.367.jar)


26/01/07 10:44:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
out_state = "s3a://us-accidents-dashboard-1445/processed/state_counts"
spark.read.parquet(out_state).show(5)


26/01/07 10:44:29 WARN SDKV2Upgrade: Directly referencing AWS SDK V1 credential provider com.amazonaws.auth.DefaultAWSCredentialsProviderChain. AWS SDK V1 credential providers will be removed once S3A is upgraded to SDK V2
26/01/07 10:44:33 ERROR Executor: Exception in task 0.0 in stage 1.0 (TID 1)    
org.apache.spark.SparkException: [FAILED_READ_FILE.NO_HINT] Encountered error while reading file s3a://us-accidents-dashboard-1445/processed/state_counts/part-00000-0eeecc7e-92d7-4887-a310-8ce973563075-c000.snappy.parquet.  SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:911)
	at org.apache.spark.sql.execution.datasources.v2.FileDataSourceV2$.attachFilePath(FileDataSourceV2.scala:142)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:141)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:773)
	at org.apache.spark.sql.catalyst.expres

Py4JJavaError: An error occurred while calling o57.showString.
: org.apache.spark.SparkException: [FAILED_READ_FILE.NO_HINT] Encountered error while reading file s3a://us-accidents-dashboard-1445/processed/state_counts/part-00000-0eeecc7e-92d7-4887-a310-8ce973563075-c000.snappy.parquet.  SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:911)
	at org.apache.spark.sql.execution.datasources.v2.FileDataSourceV2$.attachFilePath(FileDataSourceV2.scala:142)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:141)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:773)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spa

In [1]:
import os, subprocess, shutil

# 你的 Java 路径（你之前提供过）
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/openjdk-17.jdk/Contents/Home"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ.get("PATH", "")

print("JAVA_HOME =", os.environ["JAVA_HOME"])
print("java =", shutil.which("java"))
print(subprocess.check_output(["java", "-version"], stderr=subprocess.STDOUT).decode("utf-8"))


JAVA_HOME = /Library/Java/JavaVirtualMachines/openjdk-17.jdk/Contents/Home
java = /Library/Java/JavaVirtualMachines/openjdk-17.jdk/Contents/Home/bin/java
openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment Homebrew (build 17.0.17+0)
OpenJDK 64-Bit Server VM Homebrew (build 17.0.17+0, mixed mode, sharing)



In [2]:
import os
os.environ.pop("PYSPARK_SUBMIT_ARGS", None)
print("PYSPARK_SUBMIT_ARGS =", os.environ.get("PYSPARK_SUBMIT_ARGS"))


PYSPARK_SUBMIT_ARGS = None


In [3]:
from pyspark.sql import SparkSession

packages = ",".join([
    "org.apache.hadoop:hadoop-aws:3.4.2",
    "com.amazonaws:aws-java-sdk-bundle:1.12.367"
])

spark = (
    SparkSession.builder
    .appName("us-accidents-s3")
    .config("spark.jars.packages", packages)
    .config("spark.jars.ivy", "/Users/josiejiang/.ivy2_spark")

    # S3A 必需项
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")

    # 你之前踩过的时间参数：必须用毫秒整数
    .config("spark.hadoop.fs.s3a.threads.keepalivetime", "60000")
    .config("spark.hadoop.fs.s3a.multipart.purge.age", "86400000")
    .config("spark.hadoop.fs.s3a.retry.interval", "500")
    .config("spark.hadoop.fs.s3a.retry.throttle.interval", "100")
    .config("spark.hadoop.fs.s3a.connection.ttl", "300000")
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", "60000")
    .config("spark.hadoop.fs.s3a.connection.timeout", "60000")
    .config("spark.hadoop.fs.s3a.socket.timeout", "60000")

    # 关键：先关闭 vectored read，避免你那条 NoSuchMethodError 路径
    .config("spark.hadoop.fs.s3a.vectored.read.enabled", "false")
    .config("spark.hadoop.parquet.read.vectored.io.enabled", "false")

    .getOrCreate()
)

print("Spark:", spark.version)
print("Hadoop:", spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion())
print("Jars:", spark.sparkContext._jsc.sc().listJars())


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/07 10:51:29 WARN Utils: Your hostname, shousenjotekimac-pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.179 instead (on interface en0)
26/01/07 10:51:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/josiejiang/anaconda3/envs/us_accidents_dashboard/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/josiejiang/.ivy2_spark/cache
The jars for the packages stored in: /Users/josiejiang/.ivy2_spark/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-29ecb5b6-6a02-4496-b1fb-f50466ef065d;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.4.2 in central
	found software.amazon.awssdk#bundle;2.29.52 in centr

Spark: 4.1.0
Hadoop: 3.4.2
Jars: List(spark://192.168.0.179:49535/jars/org.apache.hadoop_hadoop-aws-3.4.2.jar, spark://192.168.0.179:49535/jars/com.amazonaws_aws-java-sdk-bundle-1.12.367.jar, spark://192.168.0.179:49535/jars/software.amazon.s3.analyticsaccelerator_analyticsaccelerator-s3-1.2.1.jar, spark://192.168.0.179:49535/jars/org.wildfly.openssl_wildfly-openssl-2.1.4.Final.jar, spark://192.168.0.179:49535/jars/software.amazon.awssdk_bundle-2.29.52.jar)


In [4]:
out_state = "s3a://us-accidents-dashboard-1445/processed/state_counts"
spark.read.parquet(out_state).show(5)


26/01/07 10:53:41 WARN CredentialProviderListFactory: Credentials option fs.s3a.aws.credentials.provider contains AWS v1 SDK entry com.amazonaws.auth.DefaultAWSCredentialsProviderChain
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


+-----+------+
|State| count|
+-----+------+
|   CA|113274|
|   FL| 56710|
|   TX| 37355|
|   SC| 24737|
|   NY| 22594|
+-----+------+
only showing top 5 rows
